<h1> Musikjahr </h1>

This Data Science project aims to analyze a Spotify user's music trends in the previous year and recommends new artists based on those music trends.

Install libraries related to API

In [2]:
%pip install python-dotenv
%pip install requests

Install libraries related to Data Science tasks

In [3]:
%pip install seaborn

Install library related to Spotify OAuth2

In [4]:
from dotenv import load_dotenv
import os
import base64
import requests
import urllib.parse
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer

Matplotlib is building the font cache; this may take a moment.


In [5]:
load_dotenv()

True

In [6]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = "http://localhost:3001"
scope = "user-top-read"

In [7]:
# Returns the authorization header required to query the API
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [8]:
def get_auth_code():
    auth_url = "https://accounts.spotify.com/authorize"
    
    params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "user-top-read",
    }

    auth_url = f"{auth_url}?{urllib.parse.urlencode(params)}"
    print(f"Please Authorize access by clicking on: {auth_url}")

    auth_code = input("Enter the code in the generated URL: ")
    
    return auth_code


In [9]:
# Returns new refresh token
def exchange_auth_code_for_refresh_token(auth_code):
    token_url = "https://accounts.spotify.com/api/token"
    
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri,
        "client_id": client_id,
        "client_secret": client_secret
        }

    result = requests.post(token_url, data=data)
    json_result = json.loads(result.content)
    refresh_token = json_result['refresh_token']
    
    return refresh_token

In [10]:
# Returns a new access token upon every execution
def refresh_access_token(refresh_token):
    token_url = "https://accounts.spotify.com/api/token"
    
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": client_id,
        "client_secret": client_secret
        }

    result = requests.post(token_url, data=data)
    json_result = json.loads(result.content)
    new_access_token = json_result['access_token']

    return new_access_token

In [11]:
auth_code = get_auth_code()
print(auth_code)

Please Authorize access by clicking on: https://accounts.spotify.com/authorize?client_id=9e3ab84ec3634133b2710bc672715e09&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3001&scope=user-top-read
<PyodideFuture pending cb=[WebLoop._decrement_in_progress()]>


Enter the code in the generated URL:  AQDlw9ousct6m4WrL3vWHbLW1kYWjHvlGzfdsa4OOCm4yuFXry7puMDQP2JWqU_p0gH1AHJ9SePD0uPjxE3ix2E9i9jYtE1a4aTgfPs9evYSenh3h4Wq3G7OHD2pQk6xZKCcJGswQcert7ZraK2Ft70rO_ZSO_p00DNXs0HIZ2FyHiPAYMOmsw


In [12]:
refresh_token = exchange_auth_code_for_refresh_token(auth_code.result())

In [131]:
# Returns the user's top tracks based on given time period
def get_top_tracks(refresh_token, time_period, amount):
    api_url = "https://api.spotify.com/v1/me/top/tracks"
    
    params = {
        "limit": amount,
        "time_range": str(time_period),
        "offset": 0
    }
    
    result = requests.get(api_url, headers=get_auth_header(refresh_access_token(refresh_token)), params=params)
    data = json.loads(result.content)

    songs_dict={"Song_Title": [], "Album": [], "Artists": [], "Song_Popularity" : []} # Add Artist_Popularity and Genre (?) (from Artist)
    
    song_data = []
    album_data = []
    artists_data = []
    popularity_data = []

    # Retrieve song name and popularity and store in respective lists
    for song in data['items']:
        song_data.append(song['name'])
        popularity_data.append(song['popularity'])

    # Retrieve album name and artist name and store in respective lists
    for song in data['items']:
        artists_data_list = song['album']['artists']
        album_data.append(song['album']['name'])
        if(len(artists_data_list) > 1):
            artists_for_song = []
            for artists in artists_data_list:
                artists_for_song.append(artists['name'])
            artists_data.append(artists_for_song)
        else:
            for artists in artists_data_list:
                artists_data.append(artists['name'])

    # Copy data from lists to dict
    for song in song_data:
        songs_dict['Song_Title'].append(song)
    for album in album_data:
        songs_dict['Album'].append(album)
    for artist in artists_data:
        songs_dict['Artists'].append(artist)
    for pop in popularity_data:
        songs_dict['Song_Popularity'].append(pop)
            
    return songs_dict


In [130]:
# Returns user's top artists
def get_top_artists():
    return top_artists_dict

In [120]:
# Returns a score of how popular the user's recent songs are
def get_listening_poularity_score(df):
    return float(df["Song_Popularity"].mean())

In [121]:
top_short_term_tracks = get_top_tracks(refresh_token, "short_term", 10) # Last 4 weeks
top_short_term_df = pd.DataFrame(top_short_term_tracks)

In [122]:
top_medium_term_tracks = get_top_tracks(refresh_token, "medium_term", 10) # Last 6 months
top_medium_term_df = pd.DataFrame(top_medium_term_tracks)

In [123]:
top_long_term_tracks = get_top_tracks(refresh_token, "long_term", 10) # Last 12 months
top_long_term_df = pd.DataFrame(top_long_term_tracks)

In [124]:
top_short_term_df

,Song_Title,Album,Artists,Song_Popularity
0,Snow On Tha Bluff,Snow On Tha Bluff,J. Cole,62
1,Rain in Cuba,Current Blues,Joe Hertz,40
2,Stick Season,Stick Season,Noah Kahan,86
3,Winter Wonderland,2014 Forest Hills Drive (10 Year Anniversary E...,J. Cole,59
4,Heavy Is The Crown (Original Score) (from the ...,Arcane League of Legends: Season 2 (Soundtrack...,"[Arcane, League of Legends]",73
5,You’re Gonna Go Far,Stick Season (We'll All Be Here Forever),Noah Kahan,80
6,Sk8er Boi,Let Go,Avril Lavigne,77
7,Remember Me (from the series Arcane League of ...,Arcane League of Legends: Season 2 (Soundtrack...,"[Arcane, League of Legends]",76
8,Soft Gentle Brilliant III,"From Silence, to Silence, and with Great Joy",Corey Kilgannon,18
9,The Line (from the series Arcane League of Leg...,Arcane League of Legends: Season 2 (Soundtrack...,"[Arcane, League of Legends]",81


In [132]:
get_listening_poularity_score(top_long_term_df)

60.3